In [1]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import ParameterGrid
import pickle

In [2]:
def sliding_window_forecast_with_scaling(df, target_col, model, model_name, model_params,
                                         window_size=25, test_ratio=0.2, drop_cols=None,
                                         log_dir="model_logs", log_filename="prediction_log.csv"):
    """
    Performs sliding window forecasting using standard scaling on both features and target.
    Inverse-transforms predictions back to original units before evaluation.
    Adds early stopping for XGBoost using test set as validation (only during tuning).
    """
    if drop_cols is None:
        drop_cols = []

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    log_path = os.path.join(log_dir, log_filename)
    log_entries = []

    df = df.dropna().sort_index()
    n = len(df)

    for start in range(0, n - window_size + 1):
        window_df = df.iloc[start:start + window_size].copy()
        X_window = window_df.drop(columns=[target_col] + drop_cols, errors='ignore')
        y_window = window_df[target_col].values

        train_size = int(len(X_window) * (1 - test_ratio))
        if train_size < 1 or train_size >= len(X_window):
            continue

        X_train = X_window.iloc[:train_size].values
        X_test = X_window.iloc[train_size:].values
        y_train = y_window[:train_size]
        y_test = y_window[train_size:]

        # Scale X
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)

        # Scale y
        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
        y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

        # Fit model on scaled X and y
        if model_name == "XGB":
            model.fit(X_train_scaled, y_train_scaled,
                      eval_set=[(X_test_scaled, y_test_scaled)], verbose=False
            )

        else:
            model.fit(X_train_scaled, y_train_scaled)

        # Predict
        y_pred_scaled = model.predict(X_test_scaled)

        # Inverse-transform prediction and test y
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
        y_test_real = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).ravel()

        # Evaluation
        mse = mean_squared_error(y_test_real, y_pred)
        mape = mean_absolute_percentage_error(y_test_real, y_pred)
        r2 = r2_score(y_test_real, y_pred)

        log_entry = {
            "model_name": model_name,
            "model_hyperparameters_dict": json.dumps(model_params),
            "window_size": window_size,
            "test_ratio": test_ratio,
            "start_date": str(df.index[start]),
            "end_date": str(df.index[start + window_size - 1]),
            "test_data_values_list": y_test_real.tolist(),
            "test_data_model_predictions_list": y_pred.tolist(),
            "MSE_score": mse,
            "MAPE_score": mape,
            "R^2_score": r2
        }
        log_entries.append(log_entry)

    log_df = pd.DataFrame(log_entries)
    if os.path.exists(log_path):
        existing = pd.read_csv(log_path)
        log_df = pd.concat([existing, log_df], ignore_index=True)
    log_df.to_csv(log_path, index=False)
    return log_df

In [3]:
def build_model(model_name, params):
    """
    Build and return a model given its type and hyperparameters.
    """
    if model_name == "LR":
        return Ridge(**params, random_state=42)
    elif model_name == "RF":
        return RandomForestRegressor(**params, random_state=42, n_jobs=-1)
    elif model_name == "SVM":
        return SVR(**params)
    elif model_name == "GB":
        return GradientBoostingRegressor(**params, random_state=42)
    elif model_name == "XGB":
        return xgb.XGBRegressor(**params, random_state=42, n_jobs=-1)
    else:
        raise ValueError(f"Unknown model type: {model_name}")

In [4]:
def tune_model(df, target_col, model_type, param_grid, window_sizes=[25, 100, 200],
               test_ratio=0.2, drop_cols=None, log_dir="model_logs"):
    """
    Performs hyperparameter tuning for a single model type over the given window sizes.
    Uses a single tqdm progress bar that counts total iterations.
    
    Returns a combined summary DataFrame with:
      - model_name
      - model_hyperparameters_dict (JSON string)
      - window_size
      - test_ratio
      - avg_MSE
      - avg_MAPE
      - avg_R^2
    """
    if drop_cols is None:
        drop_cols = []
    
    # Calculate total iterations over all window sizes and hyperparam combinations.
    total_iterations = sum([len(list(ParameterGrid(param_grid))) for _ in window_sizes])
    pbar = tqdm(total=total_iterations, desc=f"Tuning {model_type}")
    
    summaries = []
    for window in window_sizes:
        for params in ParameterGrid(param_grid):
            model = build_model(model_type, params)
            log_df = sliding_window_forecast_with_scaling(
                df=df,
                target_col=target_col,
                model=model,
                model_name=model_type,
                model_params=params,  # Pass current parameter set.
                window_size=window,
                test_ratio=test_ratio,
                drop_cols=drop_cols,
                log_dir=log_dir,
                log_filename=f"{model_type}_window_{window}.csv"
            )
            pbar.update(1)
            if log_df.empty:
                continue
            avg_mse = log_df["MSE_score"].mean()
            avg_mape = log_df["MAPE_score"].mean()
            avg_r2 = log_df["R^2_score"].mean()
            summary_entry = {
                "model_name": model_type,
                "model_hyperparameters_dict": json.dumps(params),
                "window_size": window,
                "test_ratio": test_ratio,
                "avg_MSE": avg_mse,
                "avg_MAPE": avg_mape,
                "avg_R^2": avg_r2
            }
            summaries.append(summary_entry)
    pbar.close()
    if summaries:
        combined_summary = pd.DataFrame(summaries)
        return combined_summary
    else:
        return pd.DataFrame()

In [5]:
def combine_and_top_logs(log_dir="model_logs", tops=5):
    """
    Reads all CSV files in the given log directory (matching *_window*.csv),
    combines them into a single DataFrame, groups by model_name and hyperparameter
    configuration, then computes the average MSE, MAPE, and R^2.
    Returns a DataFrame containing the top 5 configurations (by MAPE)
    for each model type.
    """
    # List all CSV files in the log directory
    all_files = [os.path.join(log_dir, f) for f in os.listdir(log_dir) if f.endswith(".csv")]
    dfs = [pd.read_csv(f) for f in all_files]
    combined_logs = pd.concat(dfs, ignore_index=True)
    
    # Group by model_name and hyperparameters and compute average metrics
    grouped_summary = combined_logs.groupby(
        ["model_name", "model_hyperparameters_dict"]
    )[["MSE_score", "MAPE_score", "R^2_score"]].mean().reset_index()
    
    # For each model, get the top five (lowest MAPE)
    top_n_list = []
    for model in grouped_summary["model_name"].unique():
        top_n = grouped_summary[grouped_summary["model_name"] == model].sort_values("MAPE_score", ascending=True).head(tops)
        top_n_list.append(top_n)
        
    top_n_combined = pd.concat(top_n_list, ignore_index=True)
    return top_n_combined

In [6]:
param_grids = {    
    "LR": {
        "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
        "solver": ["svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"],
        "tol": [1e-5, 2e-5, 5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2],
    },
    
    "RF": {
        "n_estimators": [800, 1200, 1800, 4000, 9000],
        "max_depth": [9, 10, 11],
        "min_samples_split": [2],
        "max_features": ["sqrt", "log2", 0.5],
        "bootstrap": [False],
        "criterion": ['absolute_error', 'squared_error', 'friedman_mse'],
    },
    
    "SVM": {
        "C": np.arange(0.5, 1.5 + 0.025, 0.025).tolist(),
        "epsilon": np.arange(2.5e-6, 3e-5 + 0.5e-6, 0.5e-6).tolist(),
        "kernel": ["rbf", "poly", "sigmoid", "linear"],
    },
    
    "GB": {
        "n_estimators": [800, 1200, 1800, 4000, 9000],
        "learning_rate": np.arange(0.0025, 0.02 + 0.0025, 0.0025).tolist(),
        "max_depth": [2],
        "subsample": [0.9, 0.95, 1.0],
    },
    
    "XGB": {
        "n_estimators": [4800],
        "learning_rate": [0.02, 0.025, 0.03, 0.035, 0.04],
        "max_depth": [2],
        "subsample": [0.75, 0.825, 0.9075, 1.0],
        "colsample_bytree": [0.9, 0.95, 1.0],
        "early_stopping_rounds": [20, 32, 50, 80],
    },
}

In [7]:
with open("temp_output/quarterly_X_y.pkl", "rb") as f:
    data_q = pickle.load(f)

drop_columns = []
print(data_q.keys())  # Should show all the tickers like AAPL, MSFT, etc.
quaterly_data = data_q["AAPL"]

dict_keys(['AAPL', 'MSFT', 'LLY', 'UNH', 'V', 'MA', 'GOOGL', 'META', 'AMZN', 'TSLA', 'PG', 'WMT', 'RTX', 'UNP', 'XOM', 'CVX', 'LIN', 'SHW', 'AMT', 'PLD', 'NEE', 'SO'])


In [8]:
lr_summary_q = tune_model(quaterly_data, target_col="y", model_type="LR",
                           param_grid=param_grids["LR"],
                           window_sizes=[10],
                           test_ratio=0.2, drop_cols=drop_columns,
                           log_dir="model_logs_q")

rf_summary_q = tune_model(quaterly_data, target_col="y", model_type="RF",
                          param_grid=param_grids["RF"],
                          window_sizes=[10], #[25, 100, 200],
                          test_ratio=0.2, drop_cols=drop_columns,
                          log_dir="model_logs_q")

svm_summary_q = tune_model(quaterly_data, target_col="y", model_type="SVM",
                           param_grid=param_grids["SVM"],
                           window_sizes=[10], #[25, 100, 200],
                           test_ratio=0.2, drop_cols=drop_columns,
                           log_dir="model_logs_q")

gb_summary_q = tune_model(quaterly_data, target_col="y", model_type="GB",
                          param_grid=param_grids["GB"],
                          window_sizes=[10], #[25, 100, 200],
                          test_ratio=0.2, drop_cols=drop_columns,
                          log_dir="model_logs_q")

xgb_summary_q = tune_model(quaterly_data, target_col="y", model_type="XGB",
                           param_grid=param_grids["XGB"],
                           window_sizes=[10], #[25, 100, 200],
                           test_ratio=0.2, drop_cols=drop_columns,
                           log_dir="model_logs_q")

Tuning XGB: 100%|██████████| 240/240 [46:13<00:00, 11.56s/it]


In [9]:
combined_summary_q = pd.concat([lr_summary_q, rf_summary_q, svm_summary_q, gb_summary_q, xgb_summary_q], ignore_index=True)
print("Combined Tuning Summary:")
display(combined_summary_q)

# Alternatively, read all prediction log files from log directory
top5_summary_q = combine_and_top_logs(log_dir="model_logs_q")
print("Top 5 Configurations per Model:")
display(top5_summary_q)

Combined Tuning Summary:


,model_name,model_hyperparameters_dict,window_size,test_ratio,avg_MSE,avg_MAPE,avg_R^2
0,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 1e-05}",10,0.2,2156.204406,0.196240,-257.723586
1,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 2e-05}",10,0.2,2156.204406,0.196240,-257.723586
2,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 5e-05}",10,0.2,2156.204406,0.196240,-257.723586
3,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 0.0001}",10,0.2,2156.204406,0.196240,-257.723586
4,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 0.0002}",10,0.2,2156.204406,0.196240,-257.723586
...,...,...,...,...,...,...,...
10094,XGB,"{""colsample_bytree"": 1.0, ""early_stopping_roun...",10,0.2,520.766636,0.140625,-88.883288
10095,XGB,"{""colsample_bytree"": 1.0, ""early_stopping_roun...",10,0.2,520.737991,0.140624,-88.951735
10096,XGB,"{""colsample_bytree"": 1.0, ""early_stopping_roun...",10,0.2,520.674537,0.140617,-88.983826
10097,XGB,"{""colsample_bytree"": 1.0, ""early_stopping_roun...",10,0.2,520.650774,0.140613,-88.958988


Top 5 Configurations per Model:


,model_name,model_hyperparameters_dict,MSE_score,MAPE_score,R^2_score
0,GB,"{""learning_rate"": 0.0125, ""max_depth"": 2, ""n_e...",678.746473,0.174115,-246.943434
1,GB,"{""learning_rate"": 0.0125, ""max_depth"": 2, ""n_e...",678.746473,0.174115,-246.943434
2,GB,"{""learning_rate"": 0.0125, ""max_depth"": 2, ""n_e...",678.746473,0.174115,-246.943434
3,GB,"{""learning_rate"": 0.0125, ""max_depth"": 2, ""n_e...",678.746473,0.174115,-246.943434
4,GB,"{""learning_rate"": 0.0125, ""max_depth"": 2, ""n_e...",678.746477,0.174115,-246.943433
5,LR,"{""alpha"": 10, ""solver"": ""saga"", ""tol"": 0.01}",741.297217,0.158253,-149.777400
6,LR,"{""alpha"": 10, ""solver"": ""saga"", ""tol"": 0.005}",783.007701,0.160061,-153.080790
7,LR,"{""alpha"": 10, ""solver"": ""sag"", ""tol"": 0.01}",795.988408,0.160088,-152.190699
8,LR,"{""alpha"": 10, ""solver"": ""saga"", ""tol"": 0.002}",809.679313,0.161061,-154.893137
9,LR,"{""alpha"": 10, ""solver"": ""sag"", ""tol"": 0.005}",819.102529,0.161200,-153.173129


In [10]:
with open("temp_output/daily_X_y.pkl", "rb") as f:
    data_d = pickle.load(f)
    
drop_columns = []
print(data_d.keys())  # Should show all the tickers like AAPL, MSFT, etc.
daily_data = data_d["AAPL"]
daily_data = daily_data.iloc[-128:,:]
daily_data

dict_keys(['AAPL', 'MSFT', 'LLY', 'UNH', 'V', 'MA', 'GOOGL', 'META', 'AMZN', 'TSLA', 'PG', 'WMT', 'RTX', 'UNP', 'XOM', 'CVX', 'LIN', 'SHW', 'AMT', 'PLD', 'NEE', 'SO', '^GSPC'])


,Adj Close,Close,High,Low,Open,Volume,SMA5,SMA50,SMA200,MACDLine,...,^IXIC,^DJI,^VIX,CL=F,GC=F,SI=F,^TNX,DX-Y.NYB,FedFundsRate,y
2024-07-01,216.023956,216.750000,217.509995,211.919998,212.089996,60402900,212.045334,190.963962,182.925126,6.332360,...,17879.300781,39169.519531,12.22,83.379997,2327.600098,29.299999,4.479,105.900002,5.33,219.532166
2024-07-02,219.532166,220.270004,220.380005,215.100006,216.149994,58046200,214.277829,192.070114,183.150480,6.644230,...,18028.759766,39331.851562,12.03,82.809998,2323.000000,29.353001,4.436,105.720001,5.33,220.807892
2024-07-03,220.807892,221.550003,221.550003,219.029999,220.000000,37369800,215.932269,193.185060,183.385837,6.914623,...,18188.300781,39308.000000,12.09,83.879997,2359.800049,30.548000,4.355,105.400002,5.33,225.581833
2024-07-05,225.581833,226.339996,226.449997,221.649994,221.649994,60412400,218.372064,194.374384,183.630370,7.428497,...,18352.759766,39375.871094,12.48,83.160004,2388.500000,31.388000,4.272,104.879997,5.33,227.056885
2024-07-08,227.056885,227.820007,227.850006,223.250000,227.089996,59085900,221.800546,195.551008,183.876819,7.864117,...,18403.740234,39344.789062,12.37,82.330002,2355.199951,30.618000,4.269,105.010002,5.33,227.913986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,257.916443,258.200012,258.209991,255.289993,255.490005,23234700,252.881967,235.564016,211.182570,6.067327,...,20031.130859,43297.031250,14.27,70.099998,2620.000000,29.974001,4.591,108.260002,4.33,258.735504
2024-12-26,258.735504,259.019989,260.100006,257.630005,258.190002,27237100,255.073553,236.071997,211.614168,6.300019,...,20020.359375,43325.800781,14.73,69.620003,2638.800049,30.047001,4.579,108.129997,4.33,255.309296
2024-12-27,255.309296,255.589996,258.700012,253.059998,257.829987,42355300,256.232281,236.552763,212.039086,6.137217,...,19722.029297,42992.210938,15.95,70.599998,2617.199951,29.655001,4.619,108.000000,4.33,251.923019
2024-12-30,251.923019,252.199997,253.500000,250.750000,252.229996,35557500,255.774783,236.958419,212.437766,5.669595,...,19486.789062,42573.730469,17.40,70.989998,2606.100098,29.106001,4.545,108.129997,4.33,250.144974


In [11]:
lr_summary_d = tune_model(daily_data, target_col="y", model_type="LR",
                           param_grid=param_grids["LR"],
                           window_sizes=[25], #[25, 100, 200],
                           test_ratio=0.2, drop_cols=drop_columns,
                           log_dir="model_logs_d")

rf_summary_d = tune_model(daily_data, target_col="y", model_type="RF",
                          param_grid=param_grids["RF"],
                          window_sizes=[25], #[25, 100, 200],
                          test_ratio=0.2, drop_cols=drop_columns,
                          log_dir="model_logs_d")

svm_summary_d = tune_model(daily_data, target_col="y", model_type="SVM",
                           param_grid=param_grids["SVM"],
                           window_sizes=[25], #[25, 100, 200],
                           test_ratio=0.2, drop_cols=drop_columns,
                           log_dir="model_logs_d")

gb_summary_d = tune_model(daily_data, target_col="y", model_type="GB",
                          param_grid=param_grids["GB"],
                          window_sizes=[25], #[25, 100, 200],
                          test_ratio=0.2, drop_cols=drop_columns,
                          log_dir="model_logs_d")

xgb_summary_d = tune_model(daily_data, target_col="y", model_type="XGB",
                           param_grid=param_grids["XGB"],
                           window_sizes=[25], #[25, 100, 200],
                           test_ratio=0.2, drop_cols=drop_columns,
                           log_dir="model_logs_d")

Tuning LR:  10%|▉         | 40/420 [00:05<01:00,  6.32it/s]/Users/jimyhc/Desktop/Spring 2025/STAT5398/XAI/Code/Pipeline/xaivenv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jimyhc/Desktop/Spring 2025/STAT5398/XAI/Code/Pipeline/xaivenv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jimyhc/Desktop/Spring 2025/STAT5398/XAI/Code/Pipeline/xaivenv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jimyhc/Desktop/Spring 2025/STAT5398/XAI/Code/Pipeline/xaivenv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ji

In [12]:
combined_summary_d = pd.concat([lr_summary_d, rf_summary_d, svm_summary_d, gb_summary_d, xgb_summary_d], ignore_index=True)
print("Combined Tuning Summary:")
display(combined_summary_d)

# Alternatively, read all prediction log files from log directory
top5_summary_d = combine_and_top_logs(log_dir="model_logs_d")
print("Top 5 Configurations per Model:")
display(top5_summary_d)

Combined Tuning Summary:


,model_name,model_hyperparameters_dict,window_size,test_ratio,avg_MSE,avg_MAPE,avg_R^2
0,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 1e-05}",25,0.2,247.572751,0.047913,-82.323978
1,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 2e-05}",25,0.2,247.572751,0.047913,-82.323978
2,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 5e-05}",25,0.2,247.572751,0.047913,-82.323978
3,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 0.0001}",25,0.2,247.572751,0.047913,-82.323978
4,LR,"{""alpha"": 0.0001, ""solver"": ""svd"", ""tol"": 0.0002}",25,0.2,247.572751,0.047913,-82.323978
...,...,...,...,...,...,...,...
10094,XGB,"{""colsample_bytree"": 1.0, ""early_stopping_roun...",25,0.2,17.407838,0.014372,-3.904144
10095,XGB,"{""colsample_bytree"": 1.0, ""early_stopping_roun...",25,0.2,17.408919,0.014373,-3.904485
10096,XGB,"{""colsample_bytree"": 1.0, ""early_stopping_roun...",25,0.2,17.409088,0.014373,-3.904444
10097,XGB,"{""colsample_bytree"": 1.0, ""early_stopping_roun...",25,0.2,17.408687,0.014373,-3.904373


Top 5 Configurations per Model:


,model_name,model_hyperparameters_dict,MSE_score,MAPE_score,R^2_score
0,GB,"{""learning_rate"": 0.01, ""max_depth"": 2, ""n_est...",25.647379,0.017385,-7.921332
1,GB,"{""learning_rate"": 0.02, ""max_depth"": 2, ""n_est...",26.058723,0.017389,-8.250682
2,GB,"{""learning_rate"": 0.02, ""max_depth"": 2, ""n_est...",26.062411,0.017391,-8.251323
3,GB,"{""learning_rate"": 0.02, ""max_depth"": 2, ""n_est...",26.062852,0.017391,-8.251388
4,GB,"{""learning_rate"": 0.02, ""max_depth"": 2, ""n_est...",26.062866,0.017391,-8.251389
5,LR,"{""alpha"": 100, ""solver"": ""sag"", ""tol"": 0.01}",16.390473,0.013874,-3.727226
6,LR,"{""alpha"": 100, ""solver"": ""sag"", ""tol"": 0.005}",16.454331,0.013897,-3.748401
7,LR,"{""alpha"": 100, ""solver"": ""sag"", ""tol"": 0.002}",16.524927,0.013933,-3.793388
8,LR,"{""alpha"": 100, ""solver"": ""lsqr"", ""tol"": 0.005}",16.525479,0.013937,-3.793985
9,LR,"{""alpha"": 100, ""solver"": ""lsqr"", ""tol"": 0.01}",16.507406,0.013938,-3.784161


In [13]:
top_summary_q = combine_and_top_logs(log_dir="model_logs_q", tops=25)
for d in top_summary_q['model_hyperparameters_dict']:
    print(json.loads(d))
    
print("="*100)

top_summary_d = combine_and_top_logs(log_dir="model_logs_d", tops=25)
for d in top_summary_d['model_hyperparameters_dict']:
    print(json.loads(d))

{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 9000, 'subsample': 0.9}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 9000, 'subsample': 0.95}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 4000, 'subsample': 0.95}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 4000, 'subsample': 0.9}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 1800, 'subsample': 0.95}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 1800, 'subsample': 0.9}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 1200, 'subsample': 0.95}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 1200, 'subsample': 0.9}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 800, 'subsample': 0.95}
{'learning_rate': 0.0125, 'max_depth': 2, 'n_estimators': 800, 'subsample': 0.9}
{'learning_rate': 0.0075, 'max_depth': 2, 'n_estimators': 9000, 'subsample': 0.95}
{'learning_rate': 0.0075, 'max_depth': 2, 'n_estimators': 9000, 'subsample': 0.9}
{'learning_r